# Creación de librerías, comandos y distribución de artefactos

Generar una librería o un artefacto con Python es un proceso bastante simple. Basta con generar un fichero `setup.py` en la raíz de tu proyecto con esta definición

In [ ]:
from setuptools import find_packages, setup

setup(name='myproject',
      version='1.0',
      packages=find_packages())

Para instalarlo, podemos hacerlo con el comando
```bash
python setup.py install
```

Una vez instalado, si ejecutamos el comando `pip freeze` para ver las librerías que tenemos instaladas, nos deberá aparecer `myproject==1.0`

La estructura de ficheros toda librería, por regla general sería:

```
/packaging_tutorial
  /myproject
    __init__.py
  setup.py
  LICENSE
  README.md
```

**Importante** saber que el nombre que le damos dentro de nuestro setup.py y el nombre de la carpeta no tienen porqué ser el mismo. Puede que tu librería de conflicto, o por nomenclatura quieres darle otro nombre, por ejemplo, en nuestro setup.py podemos ponerle `name='django_genericforms'` pero si a la carpeta la llamamos nada más `genericforms` cuando la importemos en nuestros scripts será llamada con `import genericforms`

## Distribución de artefactos

Para desplegar nuestras librerías en el repositorio oficial de Pypi, antiguamente, nos bastaba con tener una cuenta en Pypi y configurar nuestras credenciales en `~/.pypirc` tal que:

```
[distutils]
index-servers =
  pypi

[pypi]
username:<your_pypi_username>
password:<your_pypi_passwd>
```

Una vez hecho esto, podíamos registrar y subir nuestro código con  `python setup.py sdist upload`.

Pero actualmente, la forma oficial es con la librería `twine`, por lo que el anterior comando ha sido reemplazado por:

```
python setup.py sdist # genera el artefacto
twine upload -r pypi dist/* # sube el artefacto
```

Más información en la [documentación oficial](https://docs.python.org/3.7/distributing/index.html#distributing-index)

## Distribución de artefactos en servidores privados

En el trabajo, en el día a día, no es el caso común que podamos subir todas las librerías que creamos como software libre y público. Por lo que nuestros artefactos tendremos que desplegarlos en repositorios privados, para ello, igual que hicimos con Pypi, tendremos que configurar uno origen de datos en nuestro `~/.pypirc`, como por ejemplo:

```
[distutils]
index-servers =
  pypi
  local

[pypi]
username:<your_pypi_username>
password:<your_pypi_passwd>

[local]
repository: http://localhost:8080
username: pip
password: local
```

Para desplegar nuestras librerías en este origen de datos, usando los mismos comandos de antes, pero cambiando el origen en vez de `-r pypi` por `-r local` quedando tal que `twine upload -r local dist/*` o de la forma antigua `python setup.py sdist upload -r local`

## Instalar artefactos desde servidores privados

Una vez tenemos nuestra librería distribuida, podemos acceder a ella desde cualquier parte con el comando

```
pip install --extra-index-url http://localhost:8080/simple/ myproject
```

Si se da el caso que el servidor donde queremos bajarnos la imagen se ejecuta por HTTPS, la forma más cómoda de configurar nuestro proyecto sería editando el fichero `~/.config/pip/pip.conf`, o crearlo si no existe con una configuración tal que:


```
[global]
extra-index-url= http://localhost/simple
trusted-host = localhost
               pypi.org
```

De esta forma nos ahorramos el parámetro `extra-index-url` cuando hacemos un `pip install`. Quedando tal que:

```
pip install myproject
```


# Jugar en local

Para trastear y crear tus propios repositorios en local puedes levantar con docker tu propio servidor con estos comandos:

```
htpasswd -sc .htpasswd pip
docker run -v $(pwd)/.htpasswd:/data/.htpasswd -p 8080:8080 pypiserver/pypiserver:latest -P .htpasswd packages
```

Con el primer comando generamos un usuario para nuestro servidor. Al ejecutarlo te pedirá una contraseña, en nuestro caso pusimos `local` y utilizamos la configuración que mostrábamos al principio de este manual de nuestro `~/.pypirc`. 


